# Importación de liberías

In [ ]:
#IMPORTA LIBERIAS NECESARIAS

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

#ACTIVA OPCION PARA VER TODAS LAS COLUMNAS
pd.set_option('display.max_columns', None)


# Carga de datos

In [ ]:
#LEER EXCEL
train_data = pd.read_excel('telco_db.xlsx')

In [ ]:
#EXPLORAR PRIEMRAS 5 FILAS
train_data.head()

In [ ]:
#REVISAR TIPOS DE DATOS
train_data.dtypes

In [ ]:
#VER CANTIDAD DE FILAS Y COLUMNAS
train_data.shape

# Limpieza de datos y transformación de datos

In [ ]:
#TOTAL CHARGES ES OBJECT, SE DEBE CONVERTIR A NUMERICO
train_data['TotalCharges'] = pd.to_numeric(train_data['TotalCharges'], errors='coerce')

In [ ]:
# VERIFICAR QUE NO HAYA NULOS
train_data.isnull().sum()

In [ ]:
#ELIMINAR NULOS YA QUE REPRESENTAN MENOS DEL 2% DE LOS DATOS
train_data.dropna(inplace=True)

In [ ]:
# VERIFICAR QUE NO HAYA NULOS
train_data.isnull().sum()

In [ ]:
#VOLVER A REVISAR TIPOS DE DATOS
train_data.dtypes

In [ ]:
#DEFINIMOS LAS COLUMNAS QUE QUEREMOS TRANSFORMAR A BINARIAS
columnas_binarias = ['Partner',
                    'Dependents',
                    'PhoneService',
                    'PaperlessBilling',
                    'Churn',
                    'gender',
                    'MultipleLines',
                    'OnlineSecurity',
                    'OnlineBackup',
                    'DeviceProtection',
                    'TechSupport',
                    'StreamingTV',
                    'StreamingMovies'
                    ]

#ITERAMOS SOBRE LAS COLUMNAS Y APLICAMOS LA TRANSFORMACION
for col in columnas_binarias:
    train_data[col] = train_data[col].replace({'Yes': 1, 'No': 0, 'Female':1, 'Male':0, 'No internet service': 0 , 'No phone service': 0})

#LEEMOS ENCABEZADOS DE COLUMNAS
train_data[columnas_binarias].head()

In [ ]:
#VERIFICAMOS QUE SE HAYAN TRANSFORMADO CORRECTAMENTE
train_data[columnas_binarias].describe()

In [ ]:
#VOLVEMOS A VER LOS TIPOS DE DATOS
train_data.dtypes

In [ ]:
#SE CREAN COLUMNAS CATEGORICAS COMO VARIABLES DUMMIES
train_data = pd.get_dummies(data=train_data, columns=['InternetService', 'Contract', 'PaymentMethod'])

#VOLVEMOS A VER LOS TIPOS DE DATOS
train_data.dtypes

In [ ]:
#REVISAMOS COMO QUEDO EL DATAFRAME LUEGO DE LAS TRANSFORMACIONES
train_data.head()

# EDA

In [ ]:
#GENERAMOS ESTADISTICAS DESCRIPTIVAS POR CADA VARIABLE
train_data.describe()

In [ ]:
#VEMOS COMO ESTA DISTRIBUIDO EL TARGET
train_data['Churn'].value_counts()

In [ ]:
#ANALISIS UNIVARIADO
columna_a_analizar = 'Dependents'
sns.countplot(data=train_data, x= columna_a_analizar, hue='Churn')

In [ ]:
#CREA UNA MATRIZ DE CORRELACION
correlation_matrix = train_data.corr()
plt.figure(figsize=(30, 30))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Matriz de Correlación")
plt.savefig('correlation_matrix.png', dpi=300)
plt.show()



In [ ]:
#RAFICAMOS CORRELACIONES CON EL TARGET
plt.figure(figsize=(20,8))
train_data.corr()['Churn'].sort_values(ascending = False).plot(kind='bar')

# Pre entrenamiento

In [ ]:
#SELECCIONAMOS VARIABLES PREDICTORAS 
variables_pred = ['tenure', 'gender',	'SeniorCitizen'	,'Partner']

In [ ]:
#SEPARAMOS LOS DATOS ENTRE VARIABLES PREDICTORAS Y TARGET
X_train = train_data[variables_pred]
y_train = train_data['Churn']

In [ ]:
#GENERAMOS SET DE ENTRENAMIENTO Y TEST
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Regresion logistica

In [ ]:
#IMPORTAMOS EL MODELO
model_lr = LogisticRegression()
#ENTRENAMOS EL MODELO
model_lr.fit(X_train, y_train)
#ENVALUAMOS EL MODELO
preds_train_lr = model_lr.predict_proba(X_train)[:,1]
preds_test_lr  = model_lr.predict_proba(X_test)[:,1]
#CALCUAMOS EL RECALL EN TRAIN Y TEST
recall_train_lr = recall_score(y_train, preds_train_lr.round())
recall_test_lr = recall_score(y_test, preds_test_lr.round())
print('Recall en train: ', recall_train_lr)
print('Recall en test: ', recall_test_lr)
#CALCUAMOS EL PRECISION EN TRAIN Y TEST
precision_train_lr = precision_score(y_train, preds_train_lr.round())
precision_test_lr = precision_score(y_test, preds_test_lr.round())
print('Precision en train: ', precision_train_lr)
print('Precision en test: ', precision_test_lr)

In [ ]:
#SE MUESTRAN LOS COEFIENCTES DE CADA VARIABLE EN LA REGRESION LOGISTICA
for i in range(len(variables_pred)):
    print(variables_pred[i], model_lr.coef_[0][i])


## Matriz de confusion para train

In [ ]:
#GRAFICAMOS LA MATRIZ DE CONFUSION
sns.heatmap(confusion_matrix(y_train, model_lr.predict(X_train)), annot=True, fmt="d")
plt.xlabel("Predicho")
plt.ylabel("Real")

## Matriz de confusion para test

In [ ]:
#GRAFICAMOS LA MATRIZ DE CONFUSION
sns.heatmap(confusion_matrix(y_test, model_lr.predict(X_test)), annot=True, fmt="d")
plt.xlabel("Predicho")
plt.ylabel("Real")

# Random Forest

In [ ]:
#SET DE PROFUNDIDAD MAXIMA
profundidad_maxima = 5

In [ ]:
#IMPORTAMOS EL MODELO
model_rf = RandomForestClassifier(max_depth=profundidad_maxima)
#ENTRENAMOS EL MODELO
model_rf.fit(X_train, y_train)
#EVALUAMOS EL MODELO
preds_train_rf = model_rf.predict_proba(X_train)[:,1]
preds_test_rf  = model_rf.predict_proba(X_test)[:,1]
#CALCUAMOS EL RECALL EN TRAIN Y TEST
recall_train_rf = recall_score(y_train, preds_train_rf.round()) 
recall_test_rf = recall_score(y_test, preds_test_rf.round())
print('Recall en train: ', recall_train_rf)
print('Recall en test: ', recall_test_rf)
#CALCULAMOS LA PRECISION EN TRAIN Y TEST
precision_train_rf = precision_score(y_train, preds_train_rf.round())
precision_test_rf = precision_score(y_test, preds_test_rf.round())
print('Precision en train: ', precision_train_rf)
print('Precision en test: ', precision_test_rf)

# Matriz de confusion para train

In [ ]:
# GRAFICAMOS LA MATRIZ DE CONFUSION
sns.heatmap(confusion_matrix(y_train, model_rf.predict(X_train)), annot=True, fmt="d")
plt.xlabel("Predicho")
plt.ylabel("Real")

# Matriz de confusion para test

In [ ]:
# GRAFICAMOS LA MATRIZ DE CONFUSION
sns.heatmap(confusion_matrix(y_test, model_rf.predict(X_test)), annot=True, fmt="d")
plt.xlabel("Predicho")
plt.ylabel("Real")

# Rendimiento de campaña

In [ ]:
#SET DEL COSTO DE CONTACTO
costo_de_contacto = 10

#SET DEL CLV DEL CLIENTE
clv_cliente = 100

################################
#Para regresión logística
################################

#CALCULO DE CLIENTES CONTACTADOS
clientes_contactados = (model_lr.predict(X_test) == 1).sum()

#CALCULO DE CLIENTES NO FUGADOS
clientes_no_fugados  = (y_test == 0).sum()

#DEFINIMOS LOS CLIENTES RETENIDOS COMO LOS PREDICHOS 1 Y CON CHURN 1, ES DECIR, LO QUE SE IBAN A IR PERO CONTACTAMOS
clientes_retenidos   = ((model_lr.predict(X_test) == 1) & (y_test == 1)).sum()

#CALCULO DE COSTO TOTAL DE LA CAMPAÑA
costo_total_campana = clientes_contactados * costo_de_contacto

#CALCULO DE BENEFICIO DE LOS CLIENTES RETENIDOS
beneficio_retenidos = clientes_retenidos * clv_cliente

#CALCULO DE UTILIDADES
utilidad_sin_campana = clv_cliente * clientes_no_fugados
utilidad_con_campana = utilidad_sin_campana  + beneficio_retenidos - costo_total_campana

#ROI
ROI = (beneficio_retenidos - costo_total_campana)/costo_total_campana

print("Modelo de regresión logística")
print("")
print('Recall en test: ', round(recall_test_lr*100,1),"%")
print('Precision en test: ', round(precision_test_lr*100,1),"%")
print('Utilidad sin campaña:', utilidad_sin_campana)
print('Utilidad con campaña:', utilidad_con_campana)
print('Incremento de utilidad:', utilidad_con_campana - utilidad_sin_campana)
print('ROI:', ROI)

################################
#Para Random forest
################################

#CALCULO DE CLIENTES CONTACTADOS
clientes_contactados = (model_rf.predict(X_test) == 1).sum()

#CALCULO DE CLIENTES NO FUGADOS
clientes_no_fugados  = (y_test == 0).sum()

#DEFINIMOS LOS CLIENTES RETENIDOS COMO LOS PREDICHOS 1 Y CON CHURN 1, ES DECIR, LO QUE SE IBAN A IR PERO CONTACTAMOS
clientes_retenidos   = ((model_rf.predict(X_test) == 1) & (y_test == 1)).sum()

#CALCULO DE COSTO TOTAL DE LA CAMPAÑA
costo_total_campana = clientes_contactados * costo_de_contacto

#CALCULO DE BENEFICIO DE LOS CLIENTES RETENIDOS
beneficio_retenidos = clientes_retenidos * clv_cliente

#CALCULO DE UTILIDADES
utilidad_sin_campana = clv_cliente * clientes_no_fugados
utilidad_con_campana = utilidad_sin_campana  + beneficio_retenidos - costo_total_campana

#ROI
ROI = (beneficio_retenidos - costo_total_campana)/costo_total_campana

print("")
print("Modelo de Random Forest")
print("")
print('Recall en test: ', round(recall_test_rf*100,1),"%")
print('Precision en test: ', round(precision_test_rf*100,1),"%")
print('Utilidad sin campaña:', utilidad_sin_campana)
print('Utilidad con campaña:', utilidad_con_campana)
print('Incremento de utilidad:', utilidad_con_campana - utilidad_sin_campana)
print('ROI:', ROI)